In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import glob
import os
import numpy as np

from scipy.stats import binom

sns.set(style="white", font_scale=2.0)
palette = ["#780000", "#C1121F", "#f6ae2d", "#F78C6B", "#A6A867", "#003049", "#669BBC"]
sns.palplot(palette)
sns.set_palette(palette)
# sns.palplot(palette)
# sns.set_palette(palette)

pinned to dirs in /home/wesley_m/conformal-bayesopt/results/

In [ ]:
# keys = ["rnd", "ei", "nei", "ucb", "cei", "cnei", "cucb"]
# labels = {k: kk for k, kk in zip(keys,
#                                  [
#         "Random", "EI", "NEI", "UCB", "Conformal EI", "Conformal NEI", "Conformal UCB", "KG", "Conformal KG"
#                         ])}
# keys = [keys[4], keys[1], keys[5], keys[2], keys[6], keys[3], keys[0]]
# keys = [keys[-3], keys[-2], keys[-1], keys[1], keys[2], keys[3], keys[0]]

conf_acq_fns = ["cei", "cnei", "cucb"]

keys = ["rnd", "ucb", "tr_ucb", "cucb"]
names = ["Random", "UCB", "TR-UCB", "C-UCB"]

labels = {k: v for k, v in zip(keys, names)}
colors = {
    'cei': "#780000",
    'cucb': "#f6ae2d",
    'ei': "#F78C6B",
    'ucb': "#003049",
    'rnd': "#669BBC",
    'tr_ei': 'forestgreen',
    'tr_ucb': 'mediumseagreen',
}
print(labels)

root_dir = '../experiments/single_objective/data/'
subdirs = ['wesley-results/', 'wesley-results/v12/', 'tr_ei_baseline/', 'tr_ucb_baseline/']
tasks = ["levy", "ackley"]
bs_vals = [1, 3]

In [ ]:
!ls ../experiments/single_objective/data/tr_ucb_baseline

In [ ]:
def get_results(data_dir, prefix, metric_callback):
    results_dict = dict()
    search_expr = data_dir+prefix+"*.pt"
    print(search_expr)
    for f in glob.glob(search_expr):
        try:
            res = torch.load(f, map_location="cpu")
            res2 = metric_callback(res["results"])
            for key in keys:
                if key in res2:
                    results_dict.setdefault(key, [])
                    results_dict[key].append(res2[key])
        except RuntimeError:
            print("unable to load", f)
    return results_dict

In [ ]:
def clean_data(root, subdirs, task, batch_size, dim, metric):
    results_dict = {}
    for sdir in subdirs:
        data_dir = os.path.join(root, sdir)
        prefix = f"{task}{dim}_q{batch_size}"
        results_dict.update(get_results(data_dir, prefix, metric))
    return results_dict

In [ ]:
import pandas as pd

In [ ]:
def prep_mean_and_bands(lst, slice_ind=0, cummax=False):
    try:
        tsr = torch.tensor(lst)
    except:
        tsr = torch.stack(lst)#.t()
    if tsr.ndim > 2:
        tsr = tsr[...,slice_ind]
    df = pd.DataFrame(tsr.numpy())
    df = df.rolling(window=5, min_periods=1, axis=1).mean()
    tsr = torch.tensor(df.values)
    
    cummaxes = tsr.cummax(1)[0] if cummax else tsr

    ntrials = cummaxes.shape[0]
    l = torch.arange(ntrials)[binom.cdf(torch.arange(ntrials), ntrials, 0.5) < 0.025][-1]
    u = torch.arange(ntrials)[binom.cdf(torch.arange(ntrials), ntrials, 0.5) > 0.975][0]
    sorted_cvals = cummaxes.sort(0)[0][[l.item(), u.item()]]
    # print(l, u)
    medians = cummaxes.nanmedian(0)[0]
    return medians, sorted_cvals[0], sorted_cvals[1]

In [ ]:
batch_size = 3
panel_configs = [
    dict(
        task=f'ro_ackley',
        dim=5,
        batch_size=batch_size,
    ),
    dict(
        task=f'ro_ackley',
        dim=20,
        batch_size=batch_size,
    ),
    dict(
        task=f'ro_levy',
        dim=5,
        batch_size=batch_size,
    ),
    dict(
        task=f'ro_levy',
        dim=20,
        batch_size=batch_size,
    ),
]

for i, cfg in enumerate(panel_configs):
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    
    results_dict = clean_data(
        root_dir, subdirs, cfg['task'], cfg['batch_size'], cfg['dim'], lambda x: x["best_achieved"]
    )
    x = torch.arange(10, 51 * cfg["batch_size"] + 10, cfg["batch_size"])
    
    for key in keys:
        y_data = results_dict[key]
        zorder = 4 if key in conf_acq_fns else 3
        y_med, y_lb, y_ub = prep_mean_and_bands(y_data, cummax=True)
        ax.plot(x, y_med, label = labels[key], linewidth=4,
                   color=colors[key], zorder=zorder)
        ax.fill_between(x, y_lb, y_ub, alpha=0.2, color=colors[key])

    ax.set_xlabel("Function Evaluations")
    ax.set_ylabel("Best Achieved")
    sns.despine(ax=ax)
    
    title = f"{cfg['task']}_{cfg['dim']}_q{cfg['batch_size']}"
    # ax.set_title(title)
    
    save_path = f"./figures/{title}_best_achieved.pdf"
    # plt.savefig(save_path, bbox_inches="tight")

In [ ]:
figlegend = plt.figure(figsize=(2,2))
handles = ax.get_legend_handles_labels()
plt.legend(*handles, 
           loc ='upper left',
           fontsize=32,
           ncol=4
          )
plt.axis("off")
# plt.savefig(
#     f"./figures/bbo_benchmark_legend.pdf", bbox_inches="tight"
# )

In [ ]:
panel_configs = [
    dict(
        acq_fns=['tr_ucb'],
        task=f'ro_ackley',
        dim=10,
        batch_size=batch_size,
    ),
    dict(
        acq_fns=['tr_ucb'],
        task=f'ro_ackley',
        dim=20,
        batch_size=batch_size,
    ),
    dict(
        acq_fns=['tr_ucb'],
        task=f'ro_levy',
        dim=5,
        batch_size=batch_size,
    ),
    dict(
        acq_fns=['ucb', 'tr_ucb', 'cucb'],
        task=f'ro_levy',
        dim=20,
        batch_size=batch_size,
    ),
]

for i, cfg in enumerate(panel_configs):
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    x = torch.arange(10, 50 * cfg["batch_size"] + 10, cfg["batch_size"])
    
    alpha = (1.0 / (0.8 * x)**0.5).clamp_min(0.05)
    ax.plot(x, 1 - alpha, color = "black", linestyle="--", zorder=2, linewidth=4)
    
    results_dict = clean_data(
        root_dir, subdirs, cfg['task'], cfg['batch_size'], cfg['dim'], lambda x: x["coverage"]
    )
    for key in cfg['acq_fns']:
        cvrg_idx = 1 if key in conf_acq_fns else 0
        zorder = 4 if key in conf_acq_fns else 3
        y_mean, y_lb, y_ub = prep_mean_and_bands(results_dict[key], cvrg_idx, cummax=False)
        ax.plot(x, y_mean, label = labels[key], linewidth=4,
                  color=colors[key], zorder=zorder)
        ax.fill_between(x, y_lb, y_ub, alpha=0.2, color=colors[key])

    # ax.set_ylim((0.6, 1.))
    ax.set_xlabel("Function Evaluations")
    ax.set_ylabel("Holdout Coverage")
    sns.despine(ax=ax)
    
    title = f"{cfg['task']}_{cfg['dim']}_q{cfg['batch_size']}"
    # ax.set_title(title)
    
    save_path = f"./figures/{title}_holdout_cvrg.pdf"
    # plt.savefig(save_path, bbox_inches="tight")
    
    # ax[i].set_title(f"{cfg['task']}_{cfg['dim']}")
    # sns.despine(ax = ax[i])
    
# plt.tight_layout()
# ax[0].legend(ncol = 8, loc = "lower center", bbox_to_anchor = (2.5, -0.4))
# ax[0].legend(ncol = 3, loc = "lower center", bbox_to_anchor = (0.5, -0.5))

# ax[0].set_title("Levy-10D")
# ax[1].set_title("Levy-20D")
# ax[2].set_title("RO-Ackley-10D")
# ax[3].set_title("RO-Ackley-20D")
# plt.subplots_adjust(wspace=0.28)
# plt.savefig("singleobj_q3.pdf", bbox_inches = "tight")

In [ ]:
from scipy.stats import norm

plt.figure(figsize=(6, 5))
plt.plot(x, norm.ppf(1. - alpha / 2.), color='darkorange', linewidth=4, label=r'$\beta$')
plt.plot(x, 1 - alpha, color='blueviolet', linewidth=4, label=r'$1 - \alpha$')
plt.xlabel('Function evaluations')
plt.legend()